In [1]:
import pandas as pd
from glob import glob
import os
from persiantools.jdatetime import JalaliDate
from functools import reduce

ModuleNotFoundError: ignored

In [ ]:
# path = 'C:/Users/Sarv/Downloads/dashboard/marketwatch_raw/'
path_mod = 'C:/Users/Sarv/Downloads/dashboard/marketwatch_mod/'
path_ts = 'C:/Users/Sarv/Downloads/dashboard/marketwatch_ts/'
files=glob(path + '*.xlsx')
files_mod=glob(path_mod + '*.xlsx')

In [ ]:
def jdate_to_date(filename):
    year = filename.split('-')[1].split('_')[0]
    month = filename.split('-')[1].split('_')[1]
    day = filename.split('-')[1].split('_')[2]
    return str(JalaliDate(int(year), int(month), int(day)).to_gregorian())

In [ ]:
namad_sets = []
for file in files:
    df = pd.read_excel(file, header=2)
    df.columns = ['namad', 'stock_name','trade_num','trade_vol','trade_value'
              ,'prev_price','first_price','last_price','last_change','last_perc'
              ,'close_price','close_change','close_perc','low_price','high_price']
    namad_sets.append(set(df['namad']))

In [ ]:
namads = list(pd.Series([item for sublist in namad_sets for item in sublist]).unique())
namad_commons = list(reduce(set.intersection, namad_sets))
namad_uncommons = list(set(namads) - set(namad_commons))

In [ ]:
for file in files:
    filename=file.split('\\')[1].split('.')[0]
    df = pd.read_excel(file, header=2)
    df.columns = ['namad', 'stock_name','trade_num','trade_vol','trade_value'
              ,'prev_price','first_price','last_price','last_change','last_perc'
              ,'close_price','close_change','close_perc','low_price','high_price']
    df['jdate']=filename.split('-')[1].replace('_','/')
    df['date']=jdate_to_date(filename)
    df.sort_values(by='stock_name', inplace=True)
    df = df[df['namad'].isin(namad_commons)].to_excel(path_mod + f'{filename}_mod.xlsx', index=False)

In [ ]:
dfs_list = []
for file in files_mod:
    df=pd.read_excel(file)
    dfs_list.append(df)
pd.concat(dfs_list, axis=0).reset_index(drop=True).to_excel(path_mod + 'marketwatch_total.xlsx',index=False)
pd.concat(dfs_list, axis=0).reset_index(drop=True).to_csv(path_mod + 'marketwatch_total.csv',index=False)

In [ ]:
df_total = pd.read_csv(path_mod + 'marketwatch_total.csv')
namad_total = df_total.namad.unique()
for namad in namad_total:
    df_total[df_total['namad'] == namad].to_csv(path_ts + f'{namad}.csv', index=False, encoding='utf-8-sig')